In [1]:
! pip install transformers

     |████████████████████████████████| 769 kB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 690 kB 22.3 MB/s eta 0:00:01
  Using cached sacremoses-0.0.43.tar.gz (883 kB)
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
     |████████████████████████████████| 2.1 MB 27.8 MB/s eta 0:00:01     |███████████████▏                | 993 kB 27.8 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 2.8 MB/s  eta 0:00:01
  Created wheel for regex: filename=regex-2020.7.14-cp38-cp38-macosx_10_15_x86_64.whl size=286021 sha256=ada6221aeecf38f560037e3306f7f33fbb5ad11d75e4cdfc976e7a8a5336b39b
  Stored in directory: /Users/spencerbraun/Library/Caches/pip/wheels/c1/d2/6f/d7db2c6251fdcb9371d7d966eaddf630ac4e375f24252de399
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=904f9cc63446eb655f7213d683c6a48de06c19754194042c28263cb3b0966aee
  Stored in directory: /Users/spencerbraun/Library/Caches/pip/wheels/7b/78/f4/27d43a65043e1b7

In [7]:
import transformers
import os
import re
import numpy as np
import json

import requests
from bs4 import BeautifulSoup

In [9]:
def get_metadata():
    with open('arxiv/arxiv-metadata-oai-snapshot.json', 'r') as f:
        for line in f:
            yield line

In [12]:
relevant_cats = {
'cond-mat.dis-nn': 'Disordered Systems and Neural Networks',
'cs.AI': 'Artificial Intelligence',
'cs.CC': 'Computational Complexity',
'cs.CE': 'Computational Engineering, Finance, and Science',

'cs.CL': 'Computation and Language',
'cs.CR': 'Cryptography and Security',
'cs.CV': 'Computer Vision and Pattern Recognition',

'cs.DB': 'Databases',
'cs.DC': 'Distributed, Parallel, and Cluster Computing',
'cs.DL': 'Digital Libraries',
'cs.DM': 'Discrete Mathematics',
'cs.DS': 'Data Structures and Algorithms',
'cs.ET': 'Emerging Technologies',
'cs.FL': 'Formal Languages and Automata Theory',
'cs.GL': 'General Literature',
'cs.GR': 'Graphics',
'cs.GT': 'Computer Science and Game Theory',
'cs.IR': 'Information Retrieval',
'cs.IT': 'Information Theory',
'cs.LG': 'Machine Learning',
'cs.LO': 'Logic in Computer Science',
'cs.MA': 'Multiagent Systems',
'cs.MM': 'Multimedia',
'cs.MS': 'Mathematical Software',
'cs.NA': 'Numerical Analysis',
'cs.NE': 'Neural and Evolutionary Computing',
'cs.NI': 'Networking and Internet Architecture',
'cs.OH': 'Other Computer Science',
'cs.OS': 'Operating Systems',
'cs.PF': 'Performance',
'cs.PL': 'Programming Languages',
'cs.RO': 'Robotics',
'cs.SC': 'Symbolic Computation',
'cs.SD': 'Sound',
'cs.SE': 'Software Engineering',
'cs.SI': 'Social and Information Networks',
'cs.SY': 'Systems and Control',
'econ.EM': 'Econometrics',
'eess.AS': 'Audio and Speech Processing',
'eess.IV': 'Image and Video Processing',
'eess.SP': 'Signal Processing',
'math.AC': 'Commutative Algebra',
'math.AG': 'Algebraic Geometry',
'math.AP': 'Analysis of PDEs',
'math.AT': 'Algebraic Topology',
'math.CA': 'Classical Analysis and ODEs',
'math.CO': 'Combinatorics',
'math.CT': 'Category Theory',
'math.CV': 'Complex Variables',
'math.DG': 'Differential Geometry',
'math.DS': 'Dynamical Systems',
'math.FA': 'Functional Analysis',
'math.GM': 'General Mathematics',
'math.GN': 'General Topology',
'math.GR': 'Group Theory',
'math.GT': 'Geometric Topology',
'math.HO': 'History and Overview',
'math.IT': 'Information Theory',
'math.KT': 'K-Theory and Homology',
'math.LO': 'Logic',
'math.MG': 'Metric Geometry',
'math.MP': 'Mathematical Physics',
'math.NA': 'Numerical Analysis',
'math.NT': 'Number Theory',
'math.OA': 'Operator Algebras',
'math.OC': 'Optimization and Control',
'math.PR': 'Probability',
'math.QA': 'Quantum Algebra',
'math.RA': 'Rings and Algebras',
'math.RT': 'Representation Theory',
'math.SG': 'Symplectic Geometry',
'math.SP': 'Spectral Theory',
'math.ST': 'Statistics Theory',
'math-ph': 'Mathematical Physics',
'q-fin.CP': 'Computational Finance',
'q-fin.MF': 'Mathematical Finance',
'q-fin.RM': 'Risk Management',
'q-fin.ST': 'Statistical Finance',
'q-fin.TR': 'Trading and Market Microstructure',
'stat.AP': 'Applications',
'stat.CO': 'Computation',
'stat.ME': 'Methodology',
'stat.ML': 'Machine Learning',
'stat.OT': 'Other Statistics',
'stat.TH': 'Statistics Theory'}

In [50]:
metadata = get_metadata()
i = 0
with open('arxiv/arxiv-metadata-oai-snapshot.json', 'r') as f:
        for line in f:
            json_line = json.loads(line)
            print(json_line)
            break
# for paper in metadata:
#     if i % 343 == 0:
    
#         for k, v in json.loads(paper).items():
#             print(f'{k}: {v}')
#         break
#     i += 1

{'id': '0704.0001', 'submitter': 'Pavel Nadolsky', 'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan", 'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies', 'comments': '37 pages, 15 figures; published version', 'journal-ref': 'Phys.Rev.D76:013009,2007', 'doi': '10.1103/PhysRevD.76.013009', 'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and 

In [42]:
metadata = get_metadata()

In [74]:
categories

['math.NT', 'math.AG']

In [80]:

i = 0
j = 0
first_thousand = []
relevant_list = list(relevant_cats.keys())
with open('arxiv/ml_papers_snapshot.json', 'w') as write_file:
    with open('arxiv/arxiv-metadata-oai-snapshot.json', 'r') as f:
        for line in f:
            json_paper = json.loads(line)
            categories = json_paper['categories'][0].split(" ")

            if not any([x in relevant_list for x in categories]):
                continue
            try:
                date = re.search("^(\d+)\..*", json_paper['id']).group(1)
            except:
                date = None

            if int(date) < 1000:
                continue

            json_paper['categories'] = categories
            json_paper['date'] = int(date)
            json_paper['year'] = int(f'20{date[:2]}')



#             print(json_paper['title'])
            json.dump(json_paper, write_file, indent=2)



            i += 1
            if i == 100000:
                break
